<a href="https://colab.research.google.com/github/sophiawhe/data-visualization/blob/main/605_662DataVisualization_p4_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
!pip install pyproj
!pip install pycountry-convert
!pip install geopy

import pandas as pd
import numpy as np
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, WIKIMEDIA, CARTODBPOSITRON, STAMEN_TERRAIN, STAMEN_TONER, ESRI_IMAGERY, OSM
from bokeh.io import output_notebook, show
from pyproj import Proj, transform
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")
pd.set_option("max_columns", 30)

output_notebook()

In [2]:
countries_df=pd.read_csv('countries_general_info_historical.24-10-2021.csv')
countries_df.head()

,Name,NativeName,CallingCode,Iso3166P1Alpha2Code,Iso3166P1Alpha3Code,Iso3166P1NumericCode,Isni,"Population, 2010","Population, 2011","Population, 2012","Population, 2013","Population, 2014","Population, 2015","Population, 2016","Population, 2017","Population, 2018","Population, 2019"
0,Canada,Canada,1,CA,CAN,124,0000 0001 2238 5050,34004889,34339328,34714222,35082954,35437435,35702908,36109487,36545295,37065178,37593384
1,Japan,日本,81,JP,JPN,392,0000 0001 2364 9740,128070000,127833000,127629000,127445000,127276000,127141000,126994511,126785797,126529100,126264931
2,Norway,Kongeriket Norge,47,NO,NOR,578,0000 0001 2298 9524,4889252,4953088,5018573,5079623,5137232,5188607,5234519,5276968,5311916,5347896
3,Ireland,Eire,353,IE,IRL,372,0000 0001 2359 1198,4560155,4580084,4599533,4623816,4657740,4701957,4755335,4807388,4867316,4934040
4,Hungary,Magyarorszag,36,HU,HUN,348,0000 0001 2254 0380,10000023,9971727,9920362,9893082,9866468,9843028,9814023,9787966,9775564,9771141


In [3]:

#function to convert to alpah2 country codes and continents
def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)
print(get_continent(countries_df['Name'][0]))
countries_df['new_col'] = countries_df['Name'].apply(get_continent)
countries_df[['country_code', 'continent']] = pd.DataFrame(countries_df['new_col'].tolist(),  index=countries_df.index)
countries_df.head()

('CA', 'NA')


,Name,NativeName,CallingCode,Iso3166P1Alpha2Code,Iso3166P1Alpha3Code,Iso3166P1NumericCode,Isni,"Population, 2010","Population, 2011","Population, 2012","Population, 2013","Population, 2014","Population, 2015","Population, 2016","Population, 2017","Population, 2018","Population, 2019",new_col,country_code,continent
0,Canada,Canada,1,CA,CAN,124,0000 0001 2238 5050,34004889,34339328,34714222,35082954,35437435,35702908,36109487,36545295,37065178,37593384,"(CA, NA)",CA,NA
1,Japan,日本,81,JP,JPN,392,0000 0001 2364 9740,128070000,127833000,127629000,127445000,127276000,127141000,126994511,126785797,126529100,126264931,"(JP, AS)",JP,AS
2,Norway,Kongeriket Norge,47,NO,NOR,578,0000 0001 2298 9524,4889252,4953088,5018573,5079623,5137232,5188607,5234519,5276968,5311916,5347896,"(NO, EU)",NO,EU
3,Ireland,Eire,353,IE,IRL,372,0000 0001 2359 1198,4560155,4580084,4599533,4623816,4657740,4701957,4755335,4807388,4867316,4934040,"(IE, EU)",IE,EU
4,Hungary,Magyarorszag,36,HU,HUN,348,0000 0001 2254 0380,10000023,9971727,9920362,9893082,9866468,9843028,9814023,9787966,9775564,9771141,"(HU, EU)",HU,EU


In [4]:

#function to get longitude and latitude data from country name
geolocator = Nominatim()
def geolocate(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return (loc.latitude, loc.longitude)
    except:
        # Return missing value
        return np.nan

countries_df['lat_long'] = countries_df['Name'].apply(geolocate)
countries_df[['latitude', 'longitude']] = pd.DataFrame(countries_df['lat_long'].tolist(),  index=countries_df.index)
countries_df.head()

,Name,NativeName,CallingCode,Iso3166P1Alpha2Code,Iso3166P1Alpha3Code,Iso3166P1NumericCode,Isni,"Population, 2010","Population, 2011","Population, 2012","Population, 2013","Population, 2014","Population, 2015","Population, 2016","Population, 2017","Population, 2018","Population, 2019",new_col,country_code,continent,lat_long,latitude,longitude
0,Canada,Canada,1,CA,CAN,124,0000 0001 2238 5050,34004889,34339328,34714222,35082954,35437435,35702908,36109487,36545295,37065178,37593384,"(CA, NA)",CA,NA,"(61.0666922, -107.991707)",61.066692,-107.991707
1,Japan,日本,81,JP,JPN,392,0000 0001 2364 9740,128070000,127833000,127629000,127445000,127276000,127141000,126994511,126785797,126529100,126264931,"(JP, AS)",JP,AS,"(36.5748441, 139.2394179)",36.574844,139.239418
2,Norway,Kongeriket Norge,47,NO,NOR,578,0000 0001 2298 9524,4889252,4953088,5018573,5079623,5137232,5188607,5234519,5276968,5311916,5347896,"(NO, EU)",NO,EU,"(60.5000209, 9.0999715)",60.500021,9.099972
3,Ireland,Eire,353,IE,IRL,372,0000 0001 2359 1198,4560155,4580084,4599533,4623816,4657740,4701957,4755335,4807388,4867316,4934040,"(IE, EU)",IE,EU,"(52.865196, -7.9794599)",52.865196,-7.979460
4,Hungary,Magyarorszag,36,HU,HUN,348,0000 0001 2254 0380,10000023,9971727,9920362,9893082,9866468,9843028,9814023,9787966,9775564,9771141,"(HU, EU)",HU,EU,"(47.1817585, 19.5060937)",47.181759,19.506094


In [5]:
countries_df[['latitude', 'longitude']] = pd.DataFrame(countries_df['lat_long'].tolist(),  index=countries_df.index)
countries_df = countries_df.rename(columns={"Population, 2019": "population2019", "Population, 2018": "population2018"})
countries_df.head()

,Name,NativeName,CallingCode,Iso3166P1Alpha2Code,Iso3166P1Alpha3Code,Iso3166P1NumericCode,Isni,"Population, 2010","Population, 2011","Population, 2012","Population, 2013","Population, 2014","Population, 2015","Population, 2016","Population, 2017",population2018,population2019,new_col,country_code,continent,lat_long,latitude,longitude
0,Canada,Canada,1,CA,CAN,124,0000 0001 2238 5050,34004889,34339328,34714222,35082954,35437435,35702908,36109487,36545295,37065178,37593384,"(CA, NA)",CA,NA,"(61.0666922, -107.991707)",61.066692,-107.991707
1,Japan,日本,81,JP,JPN,392,0000 0001 2364 9740,128070000,127833000,127629000,127445000,127276000,127141000,126994511,126785797,126529100,126264931,"(JP, AS)",JP,AS,"(36.5748441, 139.2394179)",36.574844,139.239418
2,Norway,Kongeriket Norge,47,NO,NOR,578,0000 0001 2298 9524,4889252,4953088,5018573,5079623,5137232,5188607,5234519,5276968,5311916,5347896,"(NO, EU)",NO,EU,"(60.5000209, 9.0999715)",60.500021,9.099972
3,Ireland,Eire,353,IE,IRL,372,0000 0001 2359 1198,4560155,4580084,4599533,4623816,4657740,4701957,4755335,4807388,4867316,4934040,"(IE, EU)",IE,EU,"(52.865196, -7.9794599)",52.865196,-7.979460
4,Hungary,Magyarorszag,36,HU,HUN,348,0000 0001 2254 0380,10000023,9971727,9920362,9893082,9866468,9843028,9814023,9787966,9775564,9771141,"(HU, EU)",HU,EU,"(47.1817585, 19.5060937)",47.181759,19.506094


In [6]:
wikimedia = get_provider(CARTODBPOSITRON)

inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')

world_lon1, world_lat1 = transform(outProj,inProj,-180,-85)
world_lon2, world_lat2 = transform(outProj,inProj,180,85)

lons, lats = [], []
for lon, lat in list(zip(countries_df["longitude"], countries_df["latitude"])):
    x, y = transform(outProj,inProj,lon,lat)
    lons.append(x)
    lats.append(y)

countries_df["MercatorX"] = lons
countries_df["MercatorY"] = lats

p = figure(plot_width=900, plot_height=700,
           x_range=(world_lon1, world_lon2), y_range=(world_lat1, world_lat2),
           x_axis_type="mercator", y_axis_type="mercator",
           tooltips=[
                    ("Population", "@population2019"),("Country", "@Name"), ("(Long, Lat)", "(@longitude, @latitude)")
                    ],
           title="World Population")

p.add_tile(wikimedia)

p.circle(x="MercatorX", y="MercatorY",
         size=5,
         fill_color="dodgerblue", line_color="dodgerblue",
         fill_alpha=0.5,
         source=countries_df)

show(p)